In [2]:
import numpy as np
from tqdm import tqdm
import pandas as pd

In [3]:
#upload data
data = pd.read_csv('cheetah.cs.fiu.edu-110108-113008.1.blkparse', header=None)
#remove heading
data = data[3:]
#split columns
df = pd.DataFrame(data[0].str.split(' ',9).tolist())
#extract one column --> blocktrace sequence
blocktrace = df[3].tolist()
#convert element to int from str
blocktrace = np.array([int(x) for x in blocktrace])

In [14]:
len(blocktrace)

1322897

In [15]:
len(set(blocktrace))

671732

In [16]:
len(set(blocktrace)) / len(blocktrace)

0.5077734698922138

In [4]:
frame = 3000

## FIFO

In [18]:
cache = np.zeros((frame), dtype=int)
hit, miss = 0, 0

#print("block", "\t\t hit", "\t\t miss", "\t\t cache", "\t\tpointer")

pointer = 0
for block in tqdm(blocktrace):
    if block in cache:
        hit += 1

    elif frame - np.count_nonzero(cache) != 0:
        cache[np.where(cache==0)[0][0]] = block
        miss += 1

    else:
        cache[pointer % frame] = block
        pointer += 1
        miss += 1

    #print(block, "\t\t", hit, "\t\t", miss, "\t\t", cache, "\t\t", pointer)

hitrate = hit / (hit + miss)
print(hitrate)

100%|█████████████████████████████████████████████████████████████████████| 1322897/1322897 [00:16<00:00, 78406.39it/s]


0.05459684314047125


## OPTIMAL

In [5]:
def checkindexnp(item, nparray):

    try:
        return np.where(nparray==item)[0][0]
    except IndexError:
        return len(nparray)+1

In [6]:
cache = np.zeros((frame), dtype=int)
hit, miss = 0, 0

#print("block", "\t\t hit", "\t\t miss", "\t\t cache")
i = 0
for block in tqdm(blocktrace):
    if block in cache:
        hit += 1

    elif frame - np.count_nonzero(cache) != 0:
        cache[np.where(cache==0)[0][0]] = block
        miss += 1

    else:
        
        remain_block_trace = blocktrace[i:]
        indexed = [checkindexnp(j, remain_block_trace) for j in cache]
        cache[indexed.index(max(indexed))] = block
        miss += 1
    
    i += 1
    #print(block, "\t\t", hit, "\t\t", miss, "\t\t", cache)

hitrate = hit / (hit + miss)
print("hitrate: ", hitrate)

100%|█████████████████████████████████████████████████████████████████████| 1322897/1322897 [00:16<00:00, 80017.09it/s]


hitrate:  0.34368847079413695


In [12]:
cache = np.zeros((frame), dtype=int)
hashkey = {}
hit, miss = 0, 0

for i, block in enumerate(tqdm(blocktrace)):
    if block in hashkey:
        hit += 1
        
    elif frame - np.count_nonzero(cache) != 0:
        cache[np.where(cache==0)[0][0]] = block
        hashkey[block] = 1
        miss += 1
    
    else:
        remain_block_trace = blocktrace[i:]
        indexed = [checkindexnp(j, remain_block_trace) for j in hashkey]
        b = indexed.index(max(indexed))
        hashkey.pop(cache[b])
        cache[b] = block
        miss += 1
    
hitrate = hit / (hit + miss)
print(hitrate)

  0%|▏                                                                         | 3593/1322897 [00:04<27:36, 796.33it/s]


KeyError: 283221696

In [ ]:
l = [1,2,3,4,3,4,5,3,5,6,7]
l2 = np.array(l)

array([ True,  True,  True, ...,  True,  True,  True])

In [5]:
blocktrace[34:]

array([283193184, 283193184, 283193208, ...,  74832032,  74836592,
          748320])

In [8]:
l[10]

7